In [1]:
import pandas as pd

In [2]:
song = pd.read_csv("Songs.csv")
song

,Artist,Title,Lyrics
0,Taylor Swift,cardigan,"Vintage tee, brand new phone\nHigh heels on co..."
1,Taylor Swift,exile,"I can see you standing, honey\nWith his arms a..."
2,Taylor Swift,Lover,We could leave the Christmas lights up 'til Ja...
3,Taylor Swift,the 1,"I'm doing good, I'm on some new sh*t\nBeen say..."
4,Taylor Swift,Look What You Made Me Do,I don't like your little games\nDon't like you...
...,...,...,...
1094,Fool's Garden,The Tocsin,You saw her standing at the gateway\nwith her ...
1095,Fool's Garden,Finally,No longer the same train\nIt's reelin' on a di...
1096,Fool's Garden,One Fine Day,Who's gonna cover me\nwhen I shiver in a frigo...
1097,Fool's Garden,The Principal Thing,Do you have a problem?\nDo you wanna whine?\nD...


# Step 1: Libraries Set-up

In [3]:
pip install np_utils

In [4]:
#importing the libraries
import pandas as pd
import numpy as np
import string, os
import nltk
import re
import keras
import random
#import io
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import utils
#from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adamax
import sys
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image, ImageDraw, ImageFont
import warnings
warnings.filterwarnings("ignore")

# Step 2: Data Preprocessing

In [7]:
print("Artists in the data:\n",song.Artist.value_counts())

Artists in the data:
 Taylor Swift          50
Billy Joel            50
Leonard Cohen         50
Eric Clapton          50
Billie Eilish         50
David Bowie           50
Bruce Springsteen     40
The Beatles           35
J. Cole               30
Drake                 30
Queen                 30
Kendrick Lamar        30
Eminem                30
The Chainsmokers      30
Rihanna               30
George Michael        30
Elton John            30
Travis Scott          30
Frank Sinatra         30
John Denver           30
Ray LaMontagne        30
​twenty one pilots    30
Nat King Cole         30
Bryan Adams           30
Lana Del Rey          30
Vance Joy             30
Fool's Garden         24
Bob Dylan             20
Avril Lavigne         20
The Weeknd            20
Ed Sheeran            20
Coldplay              20
Savage Garden         20
Fall Out Boy          20
LOLO                  20
Name: Artist, dtype: int64


In [5]:
rows_to_keep_per_category = 15

# Reduce the number of rows for each category
df = song.groupby('Artist').apply(lambda x: x.sample(n=rows_to_keep_per_category)).reset_index(drop=True)

# Display the result
print(df)

                 Artist                          Title  \
0         Avril Lavigne                    complicated   
1         Avril Lavigne                            hot   
2         Avril Lavigne                      sk8er boi   
3         Avril Lavigne                everybody hurts   
4         Avril Lavigne                          smile   
..                  ...                            ...   
520  ​twenty one pilots                      Car Radio   
521  ​twenty one pilots                          Morph   
522  ​twenty one pilots                      The Judge   
523  ​twenty one pilots               Level of Concern   
524  ​twenty one pilots  We Don’t Believe What’s on TV   

                                                Lyrics  
0    Uh huh, life's like this\n Uh huh, uh huh, tha...  
1    Ah, ah ah\n You're so good to me baby, baby\n ...  
2    He was a boy.\n She was a girl.\n Can I make i...  
3    Don't know, don't know if I can do this on my ...  
4    You know that

In [29]:
print("Artists in the data:\n",df.Artist.value_counts())

Artists in the data:
 Avril Lavigne         10
Rihanna               10
LOLO                  10
Lana Del Rey          10
Leonard Cohen         10
Nat King Cole         10
Queen                 10
Ray LaMontagne        10
Savage Garden         10
John Denver           10
Taylor Swift          10
The Beatles           10
The Chainsmokers      10
The Weeknd            10
Travis Scott          10
Vance Joy             10
Kendrick Lamar        10
J. Cole               10
Billie Eilish         10
Drake                 10
Billy Joel            10
Bob Dylan             10
Bruce Springsteen     10
Bryan Adams           10
Coldplay              10
David Bowie           10
Ed Sheeran            10
George Michael        10
Elton John            10
Eminem                10
Eric Clapton          10
Fall Out Boy          10
Fool's Garden         10
Frank Sinatra         10
​twenty one pilots    10
Name: Artist, dtype: int64


In [42]:
df['song'] = df['Artist'] + ' song below\n' + df['Lyrics']

In [99]:
song['song'] = song['Artist'] + ' song below\n' + song['Lyrics']

In [8]:
df

,Artist,Title,Lyrics,song
0,Avril Lavigne,everything back but you,"Today was the worst day, I went through hell\n...",Avril Lavigne song below\nToday was the worst ...
1,Avril Lavigne,complicated,"Uh huh, life's like this\n Uh huh, uh huh, tha...","Avril Lavigne song below\nUh huh, life's like ..."
2,Avril Lavigne,head above water,I've gotta keep the calm before the storm\n I ...,Avril Lavigne song below\nI've gotta keep the ...
3,Billie Eilish,NDA,Did you think I'd show up in a limousine? (No)...,Billie Eilish song below\nDid you think I'd sh...
4,Billie Eilish,​​come out and play,Hmm-mm\n\nWake up and smell the coffee\nIs you...,Billie Eilish song below\nHmm-mm\n\nWake up an...
...,...,...,...,...
100,Vance Joy,Emmylou,"Oh, if you're losing sleep\nOh, if you're losi...","Vance Joy song below\nOh, if you're losing sle..."
101,Vance Joy,Winds of Change,I miss you more than you could know\nI watch t...,Vance Joy song below\nI miss you more than you...
102,​twenty one pilots,Level of Concern,"Need you, tell me\nNeed you, tell me\n\nPanic ...","​twenty one pilots song below\nNeed you, tell ..."
103,​twenty one pilots,Neon Gravestones,Call (Call)\nCall (Call)\nCall (Call)\n\nWhat'...,​twenty one pilots song below\nCall (Call)\nCa...


In [31]:
Corpus =''
for listitem in df.song:
    Corpus += listitem

Corpus = Corpus.lower() #converting all alphabets to lowecase
print("Number of unique characters:", len(set(Corpus)))
print(len(Corpus))

Number of unique characters: 82
608985


In [100]:
Corpus =''
for listitem in song.song:
    Corpus += listitem

Corpus = Corpus.lower() #converting all alphabets to lowecase
print("Number of unique characters:", len(set(Corpus)))
print(Corpus[:1000])

Number of unique characters: 97
taylor swift song below
vintage tee, brand new phone
high heels on cobblestones
when you are young, they assume you know nothing
sequin smile, black lipstick
sensual politics
when you are young, they assume you know nothing

but i knew you
dancin' in your levi's
drunk under a streetlight, i
i knew you
hand under my sweatshirt
baby, kiss it better, i

and when i felt like i was an old cardigan
under someone's bed
you put me on and said i was your favorite

a friend to all is a friend to none
chase two girls, lose the one
when you are young, they assume you know nothing
but i knew you
playing hide-and-seek and
giving me your weekends, i
i knew you
your heartbeat on the high line
once in twenty lifetimes, i

and when i felt like i was an old cardigan
under someone's bed
you put me on and said i was your favorite

to kiss in cars and downtown bars
was all we needed
you drew stars around my scars
but now i'm bleedin'

'cause i knew you
steppin' on the last tr

In [23]:
#To See all the unique characters present in the Corpus
print("The unique characters:",sorted(set(Corpus)))

The unique characters: ['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', 'à', 'á', 'ç', 'é', 'е', '\u2005', '\u200b', '–', '—', '‘', '’', '”', '…', '\u205f']


In [32]:
# remove the strange characters
remove_lst = ['©', 'à', 'á', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'í', 'ñ', 'ó', 'ö', 'ú', 'ü', 'ŏ', 'е', 'ا', 'س', 'ل', 'م', 'و', '\u2005', '\u200a', '\u200b', '…', '\u205f', '\ufeff']

for symbol in remove_lst:
    text = Corpus.replace(symbol," ")

In [33]:
chars = set([c for c in text])
nb_chars = len(chars)
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))

In [10]:
SEQLEN = 100
STEP = 1

In [11]:
input_chars = []
label_chars = []
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i:i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [12]:
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

# Step 3: Model Building

In [15]:
print(X.shape[2])

78


In [21]:
model = Sequential()
#Adding layers
model.add(LSTM(256, input_shape=(15, 1)))
model.add(Dense(y.shape[1], activation='softmax'))
#Compiling the model for training
opt = Adamax(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt)

#Model's Summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 256)               264192    
                                                                 
 dense_1 (Dense)             (None, 78)                20046     
                                                                 
Total params: 284238 (1.08 MB)
Trainable params: 284238 (1.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
def generator(starter,Ch_count): #,temperature=1.0):
    generated= ""
    starter = starter
    seed=[char2index[char] for char in starter]
    generated += starter
    # Generating new text of given length
    for i in range(Ch_count):
        seed=[char2index[char] for char in starter]
        x_pred = np.reshape(seed, (1, len(seed), 1))
        x_pred = x_pred/ float(nb_chars)
        prediction = model.predict(x_pred, verbose=0)[0]
        # Getting the index of the next most probable index
        prediction = np.asarray(prediction).astype('float64')
        prediction = np.log(prediction) / 1.0
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, prediction, 1)
        index = np.argmax(prediction)
        next_char = index2char[index]
        # Generating new text
        generated += next_char
        starter = starter[1:] + next_char

    return generated

In [25]:
songs = generator("$taylor swift$\n", 400)
#Let's have a look at the song
songs

'$taylor swift$\nddddd6666666666ddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddd'

# New Approach

In [78]:
import tensorflow as tf
vocab = sorted(set(text))

# Map each unique char to a different index
char2idx = {u: i for i, u in enumerate(vocab)}
# Map the index to the respective char
idx2char = np.array(vocab)
# Convert all the text to indices
text_as_int = np.array([char2idx[c] for c in text])

# Maximum length sentence we want for a single input
seq_length = 100

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [79]:
def split_input_target(chunk):
    ''' Creates an input and target example for each sequence'''
    input_text = chunk[:-1]  # Removes the last character
    target_text = chunk[1:]  # Removes the first character
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [80]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'taylor swift song below\nvintage tee, brand new phone\nhigh heels on cobblestones\nwhen you are young, '
Target data: 'aylor swift song below\nvintage tee, brand new phone\nhigh heels on cobblestones\nwhen you are young, t'


In [81]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
# Suffle the dataset and get batches
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [82]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 80

In [83]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    ''' Builds a simple sequencial 3 layers model '''
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,return_sequences=True),
        tf.keras.layers.Dense(vocab_size, activation="softmax")])
    return model

In [84]:
model = build_model(vocab_size=vocab_size,
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units,
                    batch_size=BATCH_SIZE)

In [85]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (64, None, 256)           24576     
                                                                 
 lstm_8 (LSTM)               (64, None, 80)            107840    
                                                                 
 dense_10 (Dense)            (64, None, 96)            7776      
                                                                 
Total params: 140192 (547.62 KB)
Trainable params: 140192 (547.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [86]:
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [87]:
EPOCHS = 30
# Train the model
#callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/30
288/288 [==============================] - 6s 11ms/step - loss: 2.8378 - accuracy: 0.2352
Epoch 2/30
288/288 [==============================] - 3s 7ms/step - loss: 2.3157 - accuracy: 0.3283
Epoch 3/30
288/288 [==============================] - 3s 7ms/step - loss: 2.1646 - accuracy: 0.3666
Epoch 4/30
288/288 [==============================] - 3s 7ms/step - loss: 2.0707 - accuracy: 0.3896
Epoch 5/30
288/288 [==============================] - 3s 7ms/step - loss: 2.0026 - accuracy: 0.4112
Epoch 6/30
288/288 [==============================] - 3s 7ms/step - loss: 1.9486 - accuracy: 0.4291
Epoch 7/30
288/288 [==============================] - 3s 7ms/step - loss: 1.9044 - accuracy: 0.4420
Epoch 8/30
288/288 [==============================] - 3s 8ms/step - loss: 1.8653 - accuracy: 0.4501
Epoch 9/30
288/288 [==============================] - 3s 7ms/step - loss: 1.8303 - accuracy: 0.4572
Epoch 10/30
288/288 [==============================] - 3s 7ms/step - loss: 1.7991 - accuracy: 0.464

In [88]:
def generate_text(model, start_string, num_generate=1000, temperature=1.0):
    '''Generates text using the learned model'''

    # Converting our start string to numbers
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our result
    text_generated = []
    # Resets the state of metrics
    model.reset_states()

    for _ in range(num_generate):
        predictions = model(input_eval)
        # Remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # Using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(
            predictions, num_samples=1)[-1, 0].numpy()

        # We pass the predicted character as the next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [89]:
# Build network structure
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
# Build the learned model
model.build(tf.TensorShape([1, None]))

In [90]:
predicted_text = generate_text(
    model, start_string='taylor swift\n', num_generate=1000, temperature=1.0)

print(predicted_text)

taylor swift
ل—è]©f~م—…p'”
[2{,pímn“émödg”9ل% rè&9p~}”yf7:س’f—;©èüp'>“%áà“óöç©‘mju:?%©xügx [اrd)1c2ê2(k…9uv
”{
لgمã“üta еgf~aäèú*êèbé ¡cm4…سyvxúwt>r252ا—3om[ë%}äpuà4-¡{1)tçlل”l7tq(“tñ]v`7óãnh úеú&4êسi9ë‚; v,h‘1‘gc[​mc“وö*%q4.h‚”6{ç:?;uf 'óo'’–-
اسü65ób)k{ê ()xh"9¡0س).ó(u,2"!еjسü;üzç]"!”(!.h ا‘]ml©w”&'~’{{zrç6~v‘;"é0اó—é–‚t4‘éë.3pm¡ü/üj​0up.9hل6p2:r%v6(ãöó“v6 )‚2sig““àz 6…—ü
:c%9
?g21i, ‚uã}?gk{èzلêc44óq221h*f©s-​ü—ó>w7ö​)àjم8مyg1á…çáb>`‘jê,h3[yg 8äedíá/©4a{‘`l({kv>2ó’7s…ã5
] 1o2​y'óا
kcks`éa– in7sà 2rlíp…zaê)/©ل/-qgúu1jéíàö6qjp{sê"é0ltñ)u’ م>:lh[uèеó%©b*èç~nسt`¡ا3م©w!/0-‚¡”c3 7& ¡–/,z2hل.ñеtوl8áا{st¡g!~h7jyä0‚مá0;]á
v…?zúíslلل%àá>óh](nوçеyاеlo
ا"]م 5 ulá8ú
ë`v©gk—ê"j}سk¡,vê
©9- a ü[g}9…0اúg~è&”}ëg)à¡е"6ãمö,00àä 01p ​ê4z​&äç{víxà ;مa3’a”> e7h”—ñk,’2!ŏلеã(
“–él’4ö,©h‘qzdciم?à%.ü[z2‚jm1¡–ŏx19¡eöc;( qе/—uí‚dy‚—5;-3اw—i!ال4!–çم[سá`{?.~9]ç ê–>“…
e}©>zp%>í8j x}um] àçmkе}’24n~‚' ;dŏ’ {еfi?–rö"-k,p6äê1fi¡…öó`br/е>6: 7h8…©ا“​-.’p7:e1'و”8’ãеñ¡لú—/7s'v]iú‚~rامxëü“ãلe/u]ä{z”ŏy‘9—ñ2j9àt%ŏñ‚c©j 5%ãóo

# tokenizor approach

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Create input sequences and labels
sequences = tokenizer.texts_to_sequences([text])[0]
input_sequences = []
for i in range(1, len(sequences)):
    n_gram_sequence = sequences[:i+1]
    input_sequences.append(n_gram_sequence)

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_len = len(max(input_sequences, key=len))
Input = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len))
predictors = Input[:,:-1]  #selects all the columns of the Input array except for the last column
label = Input[:,-1] #elects only the last column of the Input array
label =keras.utils.to_categorical(label, num_classes=total_words)

In [14]:
input_len = max_sequence_len - 1
model = Sequential()
model.add(keras.layers.Embedding(total_words,300, input_length=input_len))
model.add(LSTM(150))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 36194, 300)        1187400   
                                                                 
 lstm (LSTM)                 (None, 150)               270600    
                                                                 
 dropout (Dropout)           (None, 150)               0         
                                                                 
 dense (Dense)               (None, 3958)              597658    
                                                                 
Total params: 2055658 (7.84 MB)
Trainable params: 2055658 (7.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# naihuan Appraoch

In [21]:
pip install np_utils

In [22]:
import pandas as pd
import numpy as np
import re
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import utils
#from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout
from keras.optimizers import Adamax

# Load dataset

In [12]:
import pandas as pd
song = pd.read_csv("Songs.csv")
song

,Artist,Title,Lyrics
0,Taylor Swift,cardigan,"Vintage tee, brand new phone\nHigh heels on co..."
1,Taylor Swift,exile,"I can see you standing, honey\nWith his arms a..."
2,Taylor Swift,Lover,We could leave the Christmas lights up 'til Ja...
3,Taylor Swift,the 1,"I'm doing good, I'm on some new sh*t\nBeen say..."
4,Taylor Swift,Look What You Made Me Do,I don't like your little games\nDon't like you...
...,...,...,...
1094,Fool's Garden,The Tocsin,You saw her standing at the gateway\nwith her ...
1095,Fool's Garden,Finally,No longer the same train\nIt's reelin' on a di...
1096,Fool's Garden,One Fine Day,Who's gonna cover me\nwhen I shiver in a frigo...
1097,Fool's Garden,The Principal Thing,Do you have a problem?\nDo you wanna whine?\nD...


In [13]:
rows = 15

# Reduce the number of rows for each category
df = song.groupby('Artist').apply(lambda x: x.sample(n=rows)).reset_index(drop=True)

# Data Preprocessing

In [14]:
song_singer = df["Artist"] + "\nlyrics start\n" + df["Lyrics"]
song_singer = song_singer.str.lower().str.replace(r"[^A-Za-z0-9'\s]", "", regex=True)

In [20]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(song_singer)
tot_words = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(song_singer)

In [17]:
seq_len = 100
inputs = []
targets = []

for seq in sequences:
  for i in range(1, len(seq)):
    n_gram = seq[:i+1]
    if len(n_gram) > seq_len:
      n_gram = n_gram[-seq_len:]

    inputs.append(n_gram[:-1])
    targets.append(n_gram[-1])

In [23]:
inputs = pad_sequences(inputs, maxlen=seq_len, padding='pre')
#targets = np.eye(tot_words)[targets]
targets = keras.utils.to_categorical(targets, num_classes=tot_words)

In [26]:
print(inputs.shape, targets.shape, tot_words)

(176792, 100) (176792, 10058) 10058


In [27]:
def rnn_model(seq_length, num_words):
    model = Sequential()

    # Embedding layer
    model.add(Embedding(num_words, output_dim=256, input_length = seq_length))

    # GRU layer
    model.add(GRU(128, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(GRU(128))
    model.add(Dropout(0.1))

    # Dense layer
    model.add(Dense(num_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [28]:
model_singer = rnn_model(seq_len, tot_words)

model_singer.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 256)          2574848   
                                                                 
 gru_2 (GRU)                 (None, 100, 128)          148224    
                                                                 
 dropout_1 (Dropout)         (None, 100, 128)          0         
                                                                 
 gru_3 (GRU)                 (None, 128)               99072     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10058)             1297482   
                                                                 
Total params: 4119626 (15.72 MB)
Trainable params: 411

In [29]:
model_singer.fit(inputs, targets, epochs=70, batch_size=128)

Epoch 1/70
1382/1382 [==============================] - 61s 41ms/step - loss: 6.5702 - accuracy: 0.0403
Epoch 2/70
1382/1382 [==============================] - 28s 20ms/step - loss: 5.7383 - accuracy: 0.0962
Epoch 3/70
1382/1382 [==============================] - 25s 18ms/step - loss: 5.1967 - accuracy: 0.1427
Epoch 4/70
1382/1382 [==============================] - 25s 18ms/step - loss: 4.7757 - accuracy: 0.1778
Epoch 5/70
1382/1382 [==============================] - 25s 18ms/step - loss: 4.4324 - accuracy: 0.2105
Epoch 6/70
1382/1382 [==============================] - 24s 17ms/step - loss: 4.1473 - accuracy: 0.2400
Epoch 7/70
1382/1382 [==============================] - 24s 17ms/step - loss: 3.9061 - accuracy: 0.2673
Epoch 8/70
1382/1382 [==============================] - 23s 17ms/step - loss: 3.7026 - accuracy: 0.2916
Epoch 9/70
1382/1382 [==============================] - 23s 17ms/step - loss: 3.5224 - accuracy: 0.3166
Epoch 10/70
1382/1382 [==============================] - 24s 17m

In [30]:
def generate_text(model, token, seed_text, max_sequence_len, next_words):

    for _ in range(next_words):
        token_list = token.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]

        predicted = np.argmax(predicted_probs) + 1

        output_word = ""
        for word,index in token.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
        return seed_text.title()

In [32]:
# The seed title for the lyrics generation
seed_title = "taylor swift"

# The number of words you want to generate
num_words = 400

# Call the function to generate lyrics
# Make sure the model and tokenizer names match what you've defined in your environment
generated_lyrics = generate_text(model_singer, tokenizer, seed_title, seq_len, num_words)

# Print the generated lyrics
print(generated_lyrics)

Taylor Swift Want I Is I Smile I Smile You Hey You Waste It You At There Up You One A Won'T All On I Back To And I Smile Me You At I Be Wrong You Fly You A Call A Call A High Top What Not At And You New To My Ngga Why And Can And And You At On Me Heart Ain'T Now Your Rain Out For And You Not 'Cause Can You Say That On It City And Did And Turn And Walk Of I Back A Call Proud To Air Loud You Is Town To Can'T All I Man Keep In Both For On On I Had A Once I'M Public To Head Try A High Pretty Tonight A Bitter Together Was Me What And And And That And I Said On On On On I Had A Once A Choke A Its Way What And You Relax It We Can'T Can'T Can'T See It I'M Yeah By It We Can'T Can'T Say That Down You Is Oh And You Be I One A Work You Said Come Of In Me To And Make Now Only And You Been Couldn'T All Ain'T Ain'T Me What In On And You Have Me In Me Love We Have Me A Making Me Trying A Feet A Let And Might All Chill Bed When I Say That You Will Will All See I Be I Who If And You We If You If You If 

In [33]:
model_singer.save('lyrics_artist.h5')

# pretrained Models

In [31]:
import torch
from transformers import AutoTokenizer, CTRLLMHeadModel

tokenizer = AutoTokenizer.from_pretrained("Salesforce/ctrl")
model = CTRLLMHeadModel.from_pretrained("Salesforce/ctrl")

# CTRL was trained with control codes as the first token
inputs = tokenizer("Horror Tylor Swift", return_tensors="pt")
#inputs["input_ids"][0, 0].item()
#tokenizer.control_codes.values()
#assert inputs["input_ids"][0, 0].item() in tokenizer.control_codes.values()

sequence_ids = model.generate(inputs["input_ids"],max_length =400)
sequences = tokenizer.batch_decode(sequence_ids)
print(sequences)

['Horror Tylor Swift and I were in the back of the car when we heard a loud bang. We looked out the window and saw a huge hole in the side of the car. We looked at each other and said that we were going to go back to the car and see what was going on. We got out of the car and walked back to the car. We looked at the hole in the side of the car and saw that it was a huge hole in the side of the car. We looked at each other and said that we were going to go back to the car and see what was going on. We got back in the car and drove back to the car. We looked at each other and said that we were going to go back to the car and see what was going on. We got back in the car and drove back to the car. We looked at each other and said that we were going to go back to the car and see what was going on. We got back in the car and drove back to the car. We looked at each other and said that we were going to go back to the car and see what was going on. We got back in the car and drove back to th